<a href="https://colab.research.google.com/github/VincentChen2/Emotion-Detector-NAISS-/blob/main/Emotion_Detector_NAISS_Spring_2024_Test_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dataset


In [ ]:
num_of_folders = input('Input Number of Folders: ')
folder_names = []

for n in range(num_of_folders):
    try:
        folder_name = input('Input Folder Name: ')
        !unzip folder_name
        folder_names.append(folder_name)
    except:
        print('Folder Name Cannot be Found')

# Model

In [ ]:
#Imports
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import sklearn.model_selection import train_test_split

In [ ]:
a_array = []
for i in range(150) #150 is placeholder for number of entries per folder
    im = cv2.imread(f'HappyImages/IMG_{i}.png', cv2.IMREAD_GRAYSCALE) #HappyImages/ is a placeholder for folder names
    cv2.resize(im, dsize = (100, 100, 1)) #Resizes image to 100x100 grayscale
    a_array.append((im, 0))

In [ ]:
plt.imshow(a_array[60][0])

In [ ]:
bigArr = a_array # +Other Image Arrays
random.shuffle(bigArr)

In [ ]:
plt.imshow[bigArr[53][0]]

In [ ]:
X = [x[0] for x in bigArr]
Y = [y[1] for x in bigArr]

In [ ]:
i = tf.keras.layers.Input((100, 100, 1))
x1 = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')
x2 = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')(x1)
f = tf.keras.layers.Flatten(x2)
d1 - tf.keras.layers.Dense(64, activation = 'relu')(f)
d2 = tf.keras.layers.Dense(1, activation = 'relu')(d1)

model = tf.keras.models.Model(input = i, outputs = d2)